# Limpieza y Tokenizacion

- Cargamos librerias

In [1]:
import pandas as pd
import re
import json

In [9]:
#import spacy
#sp = spacy.load('es_core_news_sm')
#stop_words = sp.Defaults.stop_words
#from nltk.corpus import stopwords
#stopwords_español = stopwords.words('spanish')
#aditionalwords= [palabra for palabra in stopwords_español if palabra not in stop_words ]
#stop_words.extend(aditionalwords)

In [20]:
#Guardar Stop Words
with open('./Data/stopwords.txt', 'w') as f:
    f.write(json.dumps(stop_words))

### Cargamos Stopwords

In [2]:
#Leer Stop words como set
with open('./Data/stopwords.txt', 'r') as f:
    stopwords = json.loads(f.read())
len(stopwords)

704

# Funciones Limpieza de Texto
- Estandarizar tildes
- Revisar abreviaturas

In [3]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words             containing numbers.'''
    #text = fix_encoding(text)    
    #text = re.sub('[‘’“”…«».]', '', text)
    #text = re.sub('[\n\t]', ' ', text)
    #text = re.sub("(\\d|\\W)+", ' ', text)  # Removing special characters and digits    
    #text = text.replace("_","")

    text = text.lower()
    text = re.sub("\d", '',text)
    pattern = re.compile(r"""                  # Flag para iniciar el modo verbose
              #(?:[A-Za-z]\.|\'|[A-Za-z])+            # Hace match con abreviaciones como U.S.A.Nombre's
              (?:[A-Za-z]\.)+            # Hace match con abreviaciones como U.S.A.        
              | \w+(?:-\w+)*         # Hace match con palabras que pueden tener un guión interno
              # \$?\d+(?:\.\d+)?%?  # Hace match con dinero o porcentajes como $15.5 o 100%
              # \.\.\.              # Hace match con puntos suspensivos
              # [][.,;"'?():-_`]    # Hace match con signos de puntuación
              """, re.X)

    result = pattern.findall(text)
    #nltk.regexp_tokenize(text, pattern)
    return result
    
#text = "En los E.U. U.S.A. esa postal vale $15.50... (feo) `hola` Acoso Escolar (Bullying) en San Juan de Pasto.... J.D.A.A.S. baby Santiago's clubc slsda-sdasd-ss"

#clean_text(text)


- Cargamos Datos Originales

In [4]:
resumenes_docentes = pd.read_excel("./Data/Resumenes.xlsx", index_col=0 ,sheet_name = "Proyectos Docentes", dtype="str")
resumenes_estudiantes = pd.read_excel("./Data/Resumenes.xlsx", index_col=0,sheet_name="studiantiles y Trabajos de G", dtype="str")

#Asignamos Columnas
resumenes_docentes.columns = ["codigo", "titulo", "resumen", "estado", "id_autor", "nombre_autor", 
                     "programa", "facultad", "convocatoria", "grupo_investigacion", "linea_investigacion", "palabras_clave"]

resumenes_estudiantes.columns = ['codigo', 'titulo', 'resumen', 'estado', 'id_autor', 'nombre_autor',
       'programa', 'departamento', 'facultad', 'nombre_asesor', 'convocatoria', 'grupo_investigacion', 'linea_investigacion', 'palabras_clave']

#Eliminamos NAN
resumenes_docentes.dropna(inplace=True)
resumenes_estudiantes.dropna(inplace=True)

#Eliminamos registros no encontrados
resumenes_docentes['palabras_clave'] = resumenes_docentes['palabras_clave'].apply(lambda row: row if (row != 'No se encontraron palabras clave registradas') else " ")
resumenes_estudiantes['palabras_clave'] = resumenes_estudiantes['palabras_clave'].apply(lambda row: row if (row != 'No se encontraron palabras clave registradas') else " ")



### Corpus agregado al DF

In [5]:
#Agregamos Columna de Corpus palabras tokenizadas
resumenes_docentes['corpus'] = resumenes_docentes.apply(lambda row: row['titulo']+' '+row['resumen']+' '+' '+row['palabras_clave']+' '+row['nombre_autor']+' '+row['programa']+' '+' '+row['facultad']+' '+row['grupo_investigacion']+' '+row['linea_investigacion'],axis=1)

resumenes_estudiantes['corpus'] = resumenes_estudiantes.apply(lambda row: row['titulo']+' '+row['resumen']+' '+' '+row['palabras_clave']+' '+row['nombre_autor']+' '+row['nombre_asesor']+' '+row['programa']+' '+row['departamento']+' '+row['facultad']+' '+row['grupo_investigacion']+' '+row['linea_investigacion'],axis=1)

### Palabras Tokenizadas y Limpias

In [6]:
resumenes_docentes['palabras'] = ''
for index, row in resumenes_docentes.iterrows():
    tokens_corpus = clean_text(row['corpus'])
    row['palabras'] = [palabra for palabra in tokens_corpus if not palabra in stopwords and len(palabra) > 3]

resumenes_estudiantes['palabras'] = ''
for index, row in resumenes_estudiantes.iterrows():
    tokens_corpus = clean_text(row['corpus'])
    row['palabras'] = [palabra for palabra in tokens_corpus if not palabra in stopwords and len(palabra) > 3]

### Vocabulario del Resumen

In [7]:
resumenes_docentes['vocabulario'] = ''
for index, row in resumenes_docentes.iterrows():
    vocabulario = set(row['palabras'])
    row['vocabulario'] = list(vocabulario)

resumenes_estudiantes['vocabulario'] = ''
for index, row in resumenes_estudiantes.iterrows():
    vocabulario = set(row['palabras'])
    row['vocabulario'] = list(vocabulario)

In [1]:
# Revisamos Datos
resumenes_docentes.head(3)

NameError: name 'resumenes_docentes' is not defined

In [11]:

resumenes_estudiantes.head(3)

,codigo,titulo,resumen,estado,id_autor,nombre_autor,programa,departamento,facultad,nombre_asesor,convocatoria,grupo_investigacion,linea_investigacion,palabras_clave,corpus,palabras,vocabulario
No.,,,,,,,,,,,,,,,,,
1.0,741,"AISLAMIENTO, CARACTERIZACION, ESTABILIDAD Y AC...",El color es una de las cualidades sensoriales ...,Terminado,12324,Omaira Carolina Betancourt Ramos,Química,Química,Ciencias Exactas y Naturales,Nelson Humberto,Adicional de Trabajos de Grado 2013,GRUPO DE INVESTIGACIÓN EN PRODUCTOS DE IMPORTA...,Productos naturales,,"AISLAMIENTO, CARACTERIZACION, ESTABILIDAD Y AC...","[aislamiento, caracterizacion, estabilidad, ac...","[búsqueda, importancia, adicional, claras, est..."
2.0,760,ANÁLISIS DE ALGORITMOS PARALELOS PARA LA TAREA...,La propuesta de la investigación se centra en ...,Cancelado,12479,Rosa Maria Zambrano Burbano,Ingeniería de Sistemas,Sistemas,Ingeniería,Manuel Ernesto,Adicional de Trabajos de Grado 2013,GRIAS,Deteccione de patrones con tecnicas de mineria...,,ANÁLISIS DE ALGORITMOS PARALELOS PARA LA TAREA...,"[análisis, algoritmos, paralelos, tarea, miner...","[determinar, búsqueda, límite, importancia, eq..."
3.0,1412,ANÁLISIS DE FUNCIONALIDAD DE LA HERRAMIENTA DE...,Hoy en día se puede encontrar muchas opciones ...,Vencido,13872,Carlos Mario Coral Cabrera,Ingeniería de Sistemas,Sistemas,Ingeniería,Silvio Ricardo,Estudiantil 2017,GRIAS,Descubrimiento de conocimiento en bases de datos,Análisis de funcionalidad,ANÁLISIS DE FUNCIONALIDAD DE LA HERRAMIENTA DE...,"[análisis, funcionalidad, herramienta, intelig...","[dependencias, gestión, aplicados, herramienta..."


# Carga de datos de prueba limpios

**ejecutar a partir de aqui para los LEMAS**

- Guardar CSV

In [12]:
resumenes_docentes.to_csv(r'./Data/ResumenesDocentesLemas.csv')
resumenes_estudiantes.to_csv(r'./Data/ResumenesEstudiantesLemas.csv')


# Lematizar Palabras

### Cargar Datos ya procesados

In [1]:
import pandas as pd
import ast

resumenes_docentes = pd.read_csv("./Data/ResumenesDocentes.csv", index_col=0, dtype="str")
resumenes_estudiantes = pd.read_csv("./Data/ResumenesEstudiantes.csv", index_col=0, dtype="str")


In [3]:
resumenes_estudiantes

,codigo,titulo,resumen,estado,id_autor,nombre_autor,programa,departamento,facultad,nombre_asesor,convocatoria,grupo_investigacion,linea_investigacion,palabras_clave,corpus,palabras,vocabulario,lemas
No.,,,,,,,,,,,,,,,,,,
1.0,741,"AISLAMIENTO, CARACTERIZACION, ESTABILIDAD Y AC...",El color es una de las cualidades sensoriales ...,Terminado,12324,Omaira Carolina Betancourt Ramos,Química,Química,Ciencias Exactas y Naturales,Nelson Humberto,Adicional de Trabajos de Grado 2013,GRUPO DE INVESTIGACIÓN EN PRODUCTOS DE IMPORTA...,Productos naturales,,"AISLAMIENTO, CARACTERIZACION, ESTABILIDAD Y AC...","['aislamiento', 'caracterizacion', 'estabilida...","['dillenni', 'específico', 'cualidades', 'pare...","['dillenni', 'específico', 'cualidad', 'parece..."
2.0,760,ANÁLISIS DE ALGORITMOS PARALELOS PARA LA TAREA...,La propuesta de la investigación se centra en ...,Cancelado,12479,Rosa Maria Zambrano Burbano,Ingeniería de Sistemas,Sistemas,Ingeniería,Manuel Ernesto,Adicional de Trabajos de Grado 2013,GRIAS,Deteccione de patrones con tecnicas de mineria...,,ANÁLISIS DE ALGORITMOS PARALELOS PARA LA TAREA...,"['análisis', 'algoritmos', 'paralelos', 'tarea...","['herramientas', 'tecnología', 'deteccione', '...","['herramienta', 'tecnología', 'deteccione', 'm..."
3.0,1412,ANÁLISIS DE FUNCIONALIDAD DE LA HERRAMIENTA DE...,Hoy en día se puede encontrar muchas opciones ...,Vencido,13872,Carlos Mario Coral Cabrera,Ingeniería de Sistemas,Sistemas,Ingeniería,Silvio Ricardo,Estudiantil 2017,GRIAS,Descubrimiento de conocimiento en bases de datos,Análisis de funcionalidad,ANÁLISIS DE FUNCIONALIDAD DE LA HERRAMIENTA DE...,"['análisis', 'funcionalidad', 'herramienta', '...","['herramientas', 'excelente', 'facultad', 'mon...","['herramienta', 'excelente', 'facultar', 'moni..."
4.0,1815,ANÁLISIS DE LA CONTRIBUCIÓN SOCIOECONÓMICA DE ...,Las mujeres con frecuencia son segregadas y ac...,Terminado,14416,Ana Lucía Taimal Aza,Economía,Economía,Ciencias Económicas y Administrativas,Robert Wilson,Trabajos de Grado 2018,Coyuntura económica y social CES,Desarrollo Regional - Coyuntura Económica y So...,Contribucion Economica,ANÁLISIS DE LA CONTRIBUCIÓN SOCIOECONÓMICA DE ...,"['análisis', 'contribución', 'socioeconómica',...","['tierra', 'tecnología', 'jefe', 'regiones', '...","['tierra', 'tecnología', 'jefe', 'región', 'fr..."
5.0,1518,"ANÁLISIS DE LE EMISIÓN DE CO2 Y CH4, CAUSADA P...",El suelo ha sido fuertemente afectado por las ...,Terminado,14059,SANDRA MILENA ENRRIQUEZ DELGADO,Ingeniería Ambiental,Recursos Naturales y Sistemas Agroforestales,Ciencias Agrícolas,Jesús Antonio,Estudiantil 2017,Agroforestería y Recursos Naturales,Suelos y Agua,Cambio de uso,"ANÁLISIS DE LE EMISIÓN DE CO2 Y CH4, CAUSADA P...","['análisis', 'emisión', 'causada', 'cambio', '...","['atmosfera', 'importar', 'propiedad', 'cámara...","['atmosfera', 'importar', 'propiedad', 'cámara..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140.0,1916,USO DE LA COLORIMETRÍA TRIESTÍMULO EN LA EVALU...,Existen diversas publicaciones que han encontr...,En Ejecución,14826,Paola Andrea Mora Leiton,Química,Química,Ciencias Exactas y Naturales,Nelson Humberto,Trabajos de Grado 2019,GRUPO DE INVESTIGACIÓN EN PRODUCTOS DE IMPORTA...,Pigmentos y aceites escenciales,Betalaínas,USO DE LA COLORIMETRÍA TRIESTÍMULO EN LA EVALU...,"['colorimetría', 'triestímulo', 'evaluación', ...","['aceites', 'nutricional', 'evaluará', 'condic...","['aceite', 'nutricional', 'evaluar', 'condicio..."
141.0,1373,VALORACIÓN DEL EXTRACTO DE LOS FRUTOS DE Leand...,El estrés oxidativo (EO) ocurre por desequilib...,Terminado,13839,Luis Ignacio Gómez Cumbal,Biología,Biología,Ciencias Exactas y Naturales,Sandra Jaqueline,Trabajos de Grado 2017,Salud Pública,Prevención de Cáncer (C),Actividad antioxidante,VALORACIÓN DEL EXTRACTO DE LOS FRUTOS DE Leand...,"['valoración', 'extracto', 'frutos', 'leandra'...","['daños', 'variaciones', 'calidad', 'neutrófil...","['daño', 'variación', 'calidad', 'neutrófilo',..."
142

## Funcion de Lematizar una palabra

In [5]:
#Importar spacy para lemas
import spacy
nlp = spacy.load('es_core_news_sm')

def lemmatizer(text):  
  doc = nlp(text)
  return ' '.join([word.lemma_ for word in doc])

### Agregar Lemas a Docentes

In [6]:
resumenes_docentes['lemas'] = ''
for index, row in resumenes_docentes.iterrows():
    #if( index > 5): break
    vocabulario = ast.literal_eval(row['vocabulario'])
    row['lemas'] = [lemmatizer(palabra) for palabra in vocabulario]
    print(f'Lemas Completo Resumen {index}')

Lemas Completo Resumen 1.0
Lemas Completo Resumen 2.0
Lemas Completo Resumen 3.0
Lemas Completo Resumen 4.0
Lemas Completo Resumen 5.0
Lemas Completo Resumen 6.0
Lemas Completo Resumen 7.0
Lemas Completo Resumen 8.0
Lemas Completo Resumen 9.0
Lemas Completo Resumen 10.0
Lemas Completo Resumen 11.0
Lemas Completo Resumen 12.0
Lemas Completo Resumen 13.0
Lemas Completo Resumen 14.0
Lemas Completo Resumen 15.0
Lemas Completo Resumen 16.0
Lemas Completo Resumen 17.0
Lemas Completo Resumen 18.0
Lemas Completo Resumen 19.0
Lemas Completo Resumen 20.0
Lemas Completo Resumen 21.0
Lemas Completo Resumen 22.0
Lemas Completo Resumen 23.0
Lemas Completo Resumen 24.0
Lemas Completo Resumen 25.0
Lemas Completo Resumen 26.0
Lemas Completo Resumen 27.0
Lemas Completo Resumen 28.0
Lemas Completo Resumen 29.0
Lemas Completo Resumen 30.0
Lemas Completo Resumen 31.0
Lemas Completo Resumen 32.0
Lemas Completo Resumen 33.0
Lemas Completo Resumen 34.0
Lemas Completo Resumen 35.0
Lemas Completo Resumen 36.0
L

### Agregar Lemas a Estudiantes

In [10]:
#Estudiantes
resumenes_estudiantes['lemas'] = ''
for index, row in resumenes_estudiantes.iterrows():
    #if( index > 5): break
    vocabulario = ast.literal_eval(row['vocabulario'])
    row['lemas'] = [lemmatizer(palabra) for palabra in vocabulario]
    print(f'Lemas Completo Resumen {index}')

Lemas Completo Resumen 1.0
Lemas Completo Resumen 2.0
Lemas Completo Resumen 3.0
Lemas Completo Resumen 4.0
Lemas Completo Resumen 5.0
Lemas Completo Resumen 6.0
Lemas Completo Resumen 7.0
Lemas Completo Resumen 8.0
Lemas Completo Resumen 9.0
Lemas Completo Resumen 10.0
Lemas Completo Resumen 11.0
Lemas Completo Resumen 12.0
Lemas Completo Resumen 13.0
Lemas Completo Resumen 14.0
Lemas Completo Resumen 15.0
Lemas Completo Resumen 16.0
Lemas Completo Resumen 17.0
Lemas Completo Resumen 18.0
Lemas Completo Resumen 19.0
Lemas Completo Resumen 20.0
Lemas Completo Resumen 21.0
Lemas Completo Resumen 22.0
Lemas Completo Resumen 23.0
Lemas Completo Resumen 24.0
Lemas Completo Resumen 25.0
Lemas Completo Resumen 26.0
Lemas Completo Resumen 27.0
Lemas Completo Resumen 28.0
Lemas Completo Resumen 29.0
Lemas Completo Resumen 30.0
Lemas Completo Resumen 31.0
Lemas Completo Resumen 32.0
Lemas Completo Resumen 33.0
Lemas Completo Resumen 34.0
Lemas Completo Resumen 35.0
Lemas Completo Resumen 36.0
L

In [23]:
lemas = [ lemmatizer(palabra) for palabra in vocabulario]

In [27]:
lemas = list(lemas)